In [1]:
# 12-22年的POI数据
# 复制BLOCKS,增加年份信息
# 默认工作环境
import arcpy
from arcpy.sa import *
arcpy.env.workspace = r"E:\stu1\Projects\POI2023\POIs.gdb"

In [2]:
# 裁剪
def clip_pois_by_range(start_year, end_year, base_folder):
    """
    Clips POIs from start_year to end_year using a predefined ring.

    Args:
    - start_year: The starting year (inclusive).
    - end_year: The ending year (inclusive).
    - base_folder: The base folder where the GDBs are located.
    """
    # 定义裁剪用的环形区域路径
    ring_path = r"E:\stu1\Projects\POI2023\POI2023.gdb\ring3"
    
    for year in range(start_year, end_year + 1):
        # 定义待裁剪的POI路径和裁剪结果的路径
        poi_path = fr"{base_folder}\POIs.gdb\POI{year}"
        output_path = fr"{base_folder}\POIs.gdb\Ring3_POI{year}"
        
        # 执行裁剪操作
        arcpy.Clip_analysis(poi_path, ring_path, output_path)
        print(f"Completed clipping for POI{year}")

In [3]:
# 使用函数示例
base_folder = r"E:\stu1\Projects\POI2023"
clip_pois_by_range(12, 22, base_folder)

Completed clipping for POI12
Completed clipping for POI13
Completed clipping for POI14
Completed clipping for POI15
Completed clipping for POI16
Completed clipping for POI17
Completed clipping for POI18
Completed clipping for POI19
Completed clipping for POI20
Completed clipping for POI21
Completed clipping for POI22


In [2]:
# 定义不同功能区对应的POI类型
typecode_ranges = {
    'R': [str(num) for num in range(120300, 120305)] + # 住宅区
        ["120000"],   # 商务住宅相关
    
    'A': ["080400", "080401", "080402"] + # 度假疗养场所
        ["080600", "080601", "080602", "080603"] + # 影剧院
        ["090100", "090101", "090102"] + # 综合医院
        [f"{num:06d}" for num in range(90200, 90212)] +  # 专科医院
        ["090000", "090400", "090500"] +
        [f"{num:06d}" for num in range(130100, 130108)] +  # 政府机关
        [f"{num:06d}" for num in range(130400, 130410)] +  # 社会团体
        ["130200", "130201", "130202"] + # 外国机构
        ["130700", "130701", "130702", "130703"] + # 工商税务机构
        [f"{num:06d}" for num in range(130600, 130607)] +  # 交通车辆管理
        ["140100", "140101", "140102"] + # 博物馆
        ["140200", "140201"] + # 展览馆
        ["140300", "140400", "140500", "140600", "140800", "140900", "141000"] +
        [f"{num:06d}" for num in range(141100, 141106 )] + # 传媒机构
        [f"{num:06d}" for num in range(141200, 141208 )] + # 学校
        ["141300", "141400"],    # 公共管理与公共服务设施用地
    
    'B': [f"{num:06d}" for num in range(50100, 50124)] +  # 中餐厅
        [f"{num:06d}" for num in range(50200, 50218)] +  # 外国餐厅
        [f"{num:06d}" for num in range(60100, 60104)] +  # 商场
        [f"{num:06d}" for num in range(60400, 60416)] +  # 超级市场
        ["061000", "061001"] + # 特色商业街
        [f"{num:06d}" for num in range(100100, 100106)] +  # 宾馆酒店
        [f"{num:06d}" for num in range(160100, 160153)] +  # 银行
        [f"{num:06d}" for num in range(160400, 160409)] +  # 保险公司
        [f"{num:06d}" for num in range(160500, 160502)] + # 证券公司
        ["120201", "120203"], #楼宇     # 商业服务业设施用地
    
    'M': ["120100"] +  # 产业园区
        ["170300"] + # 工厂
        [f"{num:06d}" for num in range(170200, 170210)],   # 公司   # 工业用地
    
    'S': ["150200"] + # 火车站
        ["150104"] + # 飞机场
        ["150300"] + # 港口相关
        ["180200"] + # 收费站
        ["180300"], # 服务区 # 交通与道路设施用地
    
    'G': [f"{num:06d}" for num in range(110100, 110106)] +  # 公园广场
        [f"{num:06d}" for num in range(110200, 110211)]    # 风景名胜 # 绿地与广场用地
}

In [9]:
import arcpy

def classify_and_save_pois_by_year_and_type(start_year, end_year, typecode_ranges, base_input_folder, base_output_folder):
    """
    对指定年份范围内的POI进行分类，并根据类型代码保存至指定数据库中。

    Args:
    - start_year: 开始年份。
    - end_year: 结束年份。
    - typecode_ranges: 一个字典，键为分类名称，值为类型代码列表。
    - base_input_folder: 输入的基本文件夹路径。
    - base_output_folder: 输出的基本文件夹路径。
    """
    for year in range(start_year, end_year + 1):
        input_feature_class = fr"{base_input_folder}\POIs.gdb\Ring3_POI{year}"
        layer_name = f"inputPoiLayer_{year}"

        # 检查并删除现有图层
        if arcpy.Exists(layer_name):
            arcpy.management.Delete(layer_name)

        # 创建新的图层
        input_poi_layer = arcpy.MakeFeatureLayer_management(input_feature_class, layer_name).getOutput(0)
        
        # 添加一个新字段，用于存储转换为字符串的TYPECODE
        new_field_name = "TYPECODE_STR"
        arcpy.AddField_management(input_poi_layer, new_field_name, "TEXT")
        
        # 计算新字段的值，将TYPECODE的值转换为字符串
        arcpy.CalculateField_management(input_poi_layer, new_field_name, "!TYPECODE!", "PYTHON3")

        for category, codes in typecode_ranges.items():
            # 创建查询表达式，使用新的字符串字段
            codes_string = "', '".join([str(code) for code in codes])
            query = f"{new_field_name} IN ('{codes_string}')"
            
            try:
                # 按类型代码筛选POI
                arcpy.SelectLayerByAttribute_management(input_poi_layer, "NEW_SELECTION", query)
                
                # 保存筛选结果为新的要素类
                output_feature_class = fr"{base_output_folder}\POIs.gdb\POI{year}_{category}"
                arcpy.CopyFeatures_management(input_poi_layer, output_feature_class)
                
                print(f"分类保存完成：{output_feature_class}")
            except Exception as e:
                print(f"An error occurred while processing POI{year}_{category}: {e}")

            # 清除当前的选择
            arcpy.SelectLayerByAttribute_management(input_poi_layer, "CLEAR_SELECTION")

        # 删除用完的图层以避免名称冲突
        arcpy.management.Delete(input_poi_layer)

    print("所有分类完成")


In [10]:
base_input_folder = r"E:\stu1\Projects\POI2023"
base_output_folder = r"E:\stu1\Projects\POI2023"
start_year = 13
end_year = 22

classify_and_save_pois_by_year_and_type(start_year, end_year, typecode_ranges, base_input_folder, base_output_folder)

分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI13_R
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI13_A
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI13_B
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI13_M
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI13_S
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI13_G
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI14_R
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI14_A
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI14_B
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI14_M
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI14_S
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI14_G
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI15_R
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI15_A
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI15_B
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI15_M
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI15_S
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI15_G
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI16_R
分类保存完成：E:\stu1\Projects\POI2023\POIs.gdb\POI16_A
分类保存完成：E:\stu1\Proje

ExecuteError: ERROR 000539: Invalid field TYPECODE
Failed to execute (CalculateField).


In [6]:
# 输入的POI要素类
input_feature_class = r"E:\stu1\Projects\POI2023\POIs.gdb\POI13"

In [7]:
if arcpy.Exists("inputPoiLayer"):  # 检查图层是否存在
    arcpy.management.Delete("inputPoiLayer")  # 如果存在，则删除

# 之后再创建图层
input_poi_layer = arcpy.MakeFeatureLayer_management(input_feature_class, "inputPoiLayer").getOutput(0)

In [8]:
for category, codes in typecode_ranges.items():
    # 创建查询
    query = "typecode IN ('{}')".format("', '".join(codes))

    # 按类型代码筛选POI
    arcpy.SelectLayerByAttribute_management(input_poi_layer, "NEW_SELECTION", query)

    # 保存筛选结果为新的要素类
    output_feature_class = "E:\stu1\Projects\POI2023\POIs.gdb\POI13_{}".format(category)
    arcpy.CopyFeatures_management(input_poi_layer, output_feature_class)

    print("分类保存完成：", output_feature_class)

    # 清除当前的选择
    arcpy.SelectLayerByAttribute_management(input_poi_layer, "CLEAR_SELECTION")

print("所有分类完成")

ExecuteError: ERROR 000358: Invalid expression
Failed to execute (SelectLayerByAttribute).


In [3]:
import arcpy

def classify_and_save_pois_by_year_and_type(start_year, end_year, typecode_ranges, base_input_folder, base_output_folder):
    """
    对指定年份范围内的POI进行分类，并根据类型代码保存至指定数据库中。

    Args:
    - start_year: 开始年份。
    - end_year: 结束年份。
    - typecode_ranges: 一个字典，键为分类名称，值为类型代码列表。
    - base_input_folder: 输入的基本文件夹路径。
    - base_output_folder: 输出的基本文件夹路径。
    """
    for year in range(start_year, end_year + 1):
        input_feature_class = fr"{base_input_folder}\POIs.gdb\Ring3_POI{year}"
        
        # 检查并删除现有图层
        if arcpy.Exists("inputPoiLayer"):
            arcpy.management.Delete("inputPoiLayer")
        
        # 创建新的图层
        input_poi_layer = arcpy.MakeFeatureLayer_management(input_feature_class, "inputPoiLayer").getOutput(0)
        
        for category, codes in typecode_ranges.items():
            # 创建查询
            query = "typecode IN ('{}')".format("', '".join(codes))
            
            # 按类型代码筛选POI
            arcpy.SelectLayerByAttribute_management(input_poi_layer, "NEW_SELECTION", query)
            
            # 保存筛选结果为新的要素类
            output_feature_class = fr"{base_output_folder}\POIs.gdb\POI{year}_{category}"
            arcpy.CopyFeatures_management(input_poi_layer, output_feature_class)
            
            print("分类保存完成：", output_feature_class)
            
            # 清除当前的选择
            arcpy.SelectLayerByAttribute_management(input_poi_layer, "CLEAR_SELECTION")

    print("所有分类完成")

In [ ]:
for category, codes in typecode_ranges.items():
    # 创建查询
    query = "typecode IN ('{}')".format("', '".join(codes))

    # 按类型代码筛选POI
    arcpy.SelectLayerByAttribute_management(input_poi_layer, "NEW_SELECTION", query)

    # 保存筛选结果为新的要素类
    output_feature_class = "E:/stu1/Projects/POI2023/POI2023.gdb/poi23/POI_{}".format(category)
    arcpy.CopyFeatures_management(input_poi_layer, output_feature_class)

    print("分类保存完成：", output_feature_class)

    # 清除当前的选择
    arcpy.SelectLayerByAttribute_management(input_poi_layer, "CLEAR_SELECTION")

print("所有分类完成")

In [ ]:
# POI分类保存

In [ ]:
# 构建影响因子

In [ ]:
# 空间链接

In [ ]:
# 去从聚

In [ ]:
# 计算频数密度

In [ ]:
# 计算网络约束的核密度

In [ ]:
# 识别功能区结果

In [ ]:
# 去丛聚频数密度


In [ ]:
#

In [ ]:
# 网络约束的核密度